In [ ]:
import pandas as pd
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split


In [ ]:
catalogo = pd.read_csv("skiff_unico_2mass_gaia_corrected_qlf_plx.csv")

In [ ]:
filters_2mass = ['j_m', 'h_m', 'ks_m']
filters_gaia = ['phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag']
filters_combined = filters_gaia + filters_2mass 

In [ ]:
from itertools import combinations
combinations = list(combinations(filters_combined, 2))
lista_colores = []
for i in range(len(combinations)):
    catalogo[combinations[i][0] +" - "+ combinations[i][1]] = catalogo[combinations[i][0]] - catalogo[combinations[i][1]]
    lista_colores.append(combinations[i][0] +" - "+ combinations[i][1])

In [ ]:
catalogo = catalogo.dropna(subset=lista_colores)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(catalogo, catalogo.label, test_size=0.3)


In [ ]:
brf = BalancedRandomForestClassifier(random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from scipy.stats import randint


# Definir los hiperparámetros a ajustar
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

# Crear el estimador del modelo
rf = BalancedRandomForestClassifier(random_state=42)

# Realizar la búsqueda aleatoria de hiperparámetros
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10, cv=5)
random_search.fit(X_train[lista_colores], y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(random_search.best_params_)



In [ ]:
from sklearn.metrics import classification_report
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced

def obtener_metricas_imbalanced(y_real, y_pred):
    # Calcula las métricas de clasificación desequilibradas
    report_imbalanced = classification_report_imbalanced(y_real, y_pred)
    geometric_mean = geometric_mean_score(y_real, y_pred)

    # Imprime el informe de clasificación desequilibrado
    print("Informe de clasificación desequilibrado:")
    print(report_imbalanced)

    # Imprime la puntuación de la media geométrica
    print("Puntuación de la media geométrica: {:.2f}".format(geometric_mean))

    # Devuelve el informe de clasificación desequilibrado y la puntuación de la media geométrica
    return report_imbalanced, geometric_mean


In [ ]:
obtener_metricas_imbalanced(X_test.label,
                          random_search.predict(X_test[lista_colores]) )

In [ ]:
import joblib


# Guarda el modelo
joblib.dump(random_search.best_estimator_, 'random_forest_model.pkl')
